In [ ]:
"""
Illustrate usage of DAPPER to (interactively) run a synthetic ("twin") experiment.
"""

In [ ]:
!pip install DA-DAPPER

In [ ]:
import dapper as dpr
import dapper.da_methods as da

In [ ]:
# Load experiment setup: the hidden Markov model (HMM)
from dapper.mods.Lorenz63.sakov2012 import HMM  # isort:skip

In [ ]:
# Generate the same random numbers every time
dpr.set_seed(3000)

In [ ]:
HMM.t.T = 30  # shorten experiment

In [ ]:
# Simulate synthetic truth (xx) and noisy obs (yy)
xx, yy = HMM.simulate()

In [ ]:
# Specify a DA method configuration ("xp" for "experiment")
xp = da.EnKF('Sqrt', N=10, infl=1.02, rot=True)
# xp = da.Var3D()
# xp = da.PartFilt(N=100,reg=2.4,NER=0.3)

In [ ]:
# Assimilate yy, knowing the HMM; xx is used to assess the performance
xp.assimilate(HMM, xx, yy, liveplots=True)

In [ ]:
# Average the time series of various statistics
xp.stats.average_in_time()

In [ ]:
# Print some averages
print(xp.avrgs.tabulate(['rmse.a', 'rmv.a']))

In [ ]:
# Replay liveplotters
xp.stats.replay(speed=100)

#### Further diagnostic plots

In [ ]:
# import dapper.tools.viz as viz
# viz.plot_rank_histogram(xp.stats)
# viz.plot_err_components(xp.stats)
# viz.plot_hovmoller(xx)

#### Explore objects

In [ ]:
# print(HMM)
# print(xp)
# print(xp.stats)
# print(xp.avrgs)

#### Excercise: Why does the replay look jagged?
Hint: provide the keyword store_u=True to assimilate() to avoid this.

#### Excercise: Why does the replay only contain the blue lines?

#### Excercise: Try using
- Optimal interpolation
- The (extended) Kalman filter
- The iterative EnKS

Hint: suggested DA xp's are listed in the HMM file

#### Excercise: Run an experiment for each of these models
- LotkaVolterra
- Lorenz96
- LA
- QG